In [3]:
import os
import requests
import json
import httpx
from openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from langchain.tools import tool
from datetime import date

In [4]:
httpx_client = httpx.Client(http2=True, verify=False, timeout=10.0)

vcapservices = os.getenv('VCAP_SERVICES')
services = json.loads(vcapservices)

def is_chatservice(service):
    return service["name"] == "gen-ai-qwen3-ultra"

chat_services = filter(is_chatservice, services["genai"])
chat_credentials = list(chat_services)[0]["credentials"]

llm = ChatOpenAI(temperature=0.9, model=chat_credentials["model_name"], base_url=chat_credentials["api_base"], api_key=chat_credentials["api_key"], http_client=httpx_client)


In [5]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())


# list of tools for agent
tools = [time]

In [6]:
# initialize agent with tools and llm
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")




> Entering new AgentExecutor chain...
<think>
Okay, the user is asking for today's date. I need to use the time tool to get that information. The time tool requires an empty string as input, so I'll call it with an empty action input. Once I get the date from the tool, I can present it to the user. Let me make sure I'm following the correct format for the JSON blob.
</think>

Thought: I need to determine today's date using the available tool.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-08-13
Thought:<think>
Okay, the user asked for today's date. I used the time tool, which returned 2025-08-13. So the final answer should be that date. I need to make sure I present it correctly.
</think>

Final Answer: Today's date is 2025-08-13.

> Finished chain.
